# Task 6.1: Custom Module Lab - Building ResNet from Scratch

**Module:** 6 - Deep Learning with PyTorch  
**Time:** 2 hours  
**Difficulty:** ⭐⭐ (Intermediate)

---

## Learning Objectives

By the end of this notebook, you will:
- [ ] Understand PyTorch's `nn.Module` architecture pattern
- [ ] Implement ResNet's `BasicBlock` with skip connections
- [ ] Implement the more efficient `Bottleneck` block
- [ ] Build a complete ResNet-18 from scratch
- [ ] Test your implementation on CIFAR-10

---

## Prerequisites

- Completed: Phase 1 (Neural Network Fundamentals)
- Knowledge of: Convolutional neural networks, backpropagation

---

## Real-World Context

ResNet (Residual Networks) won the 2015 ImageNet competition and revolutionized deep learning. Before ResNet, training very deep networks (50+ layers) was nearly impossible due to vanishing gradients. 

Today, ResNet architecture is used everywhere:
- **Medical imaging**: Detecting tumors in X-rays
- **Autonomous vehicles**: Object detection systems
- **Social media**: Photo filtering and face recognition
- **Retail**: Product recognition and inventory management

Understanding how to build ResNet teaches you the foundational patterns used in almost all modern neural networks.

---

## ELI5: What is a Residual Network?

> **Imagine you're playing telephone...** 🎮
>
> In regular telephone, a message passes through many people. By the end, it's often completely different from the original. The more people in the chain, the worse it gets!
>
> Now imagine if each person could also HEAR the original message directly, and just add small changes to it. Even after 100 people, the message stays mostly intact, with useful additions.
>
> **That's exactly what skip connections do!**
>
> - The "message" is the data flowing through the network
> - Each "person" is a layer that transforms the data
> - The "skip connection" lets the original data bypass layers
> - The network only needs to learn what to ADD, not rebuild everything
>
> **In AI terms:** Skip connections let gradients flow directly backwards, preventing them from vanishing. Instead of learning F(x), the network learns F(x) + x, making training 100+ layer networks possible!

---

## Part 1: Environment Setup and PyTorch Basics

Let's start by setting up our environment and reviewing key PyTorch concepts.

In [ ]:
# Standard imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision
import torchvision.transforms as transforms

import numpy as np
import matplotlib.pyplot as plt
from typing import Type, Union, List, Optional
import time

# Check our environment
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

# Set device - on DGX Spark, we always want to use the GPU!
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"\nUsing device: {device}")

### Understanding nn.Module

Every neural network in PyTorch inherits from `nn.Module`. Think of it as a **blueprint** that gives your model superpowers:

1. **Automatic parameter tracking** - All weights are found automatically
2. **Device management** - Move to GPU with `.to(device)`
3. **Training/eval modes** - Dropout, BatchNorm behave differently
4. **State saving** - Save/load with `state_dict()`

In [ ]:
# A simple nn.Module example
class SimpleConvBlock(nn.Module):
    """A basic convolutional block: Conv -> BatchNorm -> ReLU
    
    This is the fundamental building block of most CNNs.
    """
    
    def __init__(self, in_channels: int, out_channels: int, kernel_size: int = 3):
        # ALWAYS call super().__init__() first!
        super().__init__()
        
        # Define layers as attributes
        # PyTorch automatically tracks these as "submodules"
        self.conv = nn.Conv2d(
            in_channels, out_channels, 
            kernel_size=kernel_size, 
            padding=kernel_size // 2,  # 'same' padding
            bias=False  # BatchNorm has its own bias, so we don't need conv bias
        )
        self.bn = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)  # inplace saves memory!
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """Forward pass - define how data flows through the module."""
        x = self.conv(x)
        x = self.bn(x)
        x = self.relu(x)
        return x

# Let's test it!
block = SimpleConvBlock(3, 64)
print(f"Block structure:\n{block}")
print(f"\nNumber of parameters: {sum(p.numel() for p in block.parameters()):,}")

# Test with random input
x = torch.randn(1, 3, 32, 32)  # Batch=1, Channels=3, Height=32, Width=32
y = block(x)
print(f"\nInput shape: {x.shape}")
print(f"Output shape: {y.shape}")

### What Just Happened?

1. We created a module with 3 layers: Conv2d, BatchNorm2d, ReLU
2. PyTorch automatically found all the parameters (1,792 = 3×64×3×3 + 64×2 for BatchNorm)
3. The output has 64 channels instead of 3, but same spatial dimensions (32×32)

---

## Part 2: The BasicBlock - ResNet's Foundation

Now let's build the actual ResNet blocks. The `BasicBlock` is used in ResNet-18 and ResNet-34.

### Architecture:
```
Input (x)
    |
    +---> [3x3 Conv -> BN -> ReLU -> 3x3 Conv -> BN] --+
    |                                                   |
    +------------ (skip connection) -------------------+
    |                                                   |
    +<-------------------------------------------------+
    |
    v
  [ReLU]
    |
    v
  Output
```

The key insight: **output = ReLU(F(x) + x)**

The network learns the "residual" F(x), which is what needs to be ADDED to x.

In [ ]:
class BasicBlock(nn.Module):
    """
    ResNet BasicBlock: Two 3x3 convolutions with a skip connection.
    
    Used in ResNet-18 and ResNet-34.
    
    Args:
        in_channels: Number of input channels
        out_channels: Number of output channels
        stride: Stride for the first convolution (2 for downsampling)
    
    ELI5: This is like a highway exit ramp. The main road (skip connection)
    lets you bypass the city (conv layers) if you want. But the city might
    add something useful, so we combine both paths!
    """
    
    # Expansion factor - BasicBlock doesn't expand channels
    expansion = 1
    
    def __init__(
        self, 
        in_channels: int, 
        out_channels: int, 
        stride: int = 1
    ):
        super().__init__()
        
        # First conv: may downsample if stride=2
        self.conv1 = nn.Conv2d(
            in_channels, out_channels, 
            kernel_size=3, stride=stride, padding=1, bias=False
        )
        self.bn1 = nn.BatchNorm2d(out_channels)
        
        # Second conv: always stride=1
        self.conv2 = nn.Conv2d(
            out_channels, out_channels,
            kernel_size=3, stride=1, padding=1, bias=False
        )
        self.bn2 = nn.BatchNorm2d(out_channels)
        
        # Skip connection - only needed when dimensions change
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            # Use 1x1 conv to match dimensions
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # Save input for skip connection
        identity = x
        
        # Main path: Conv -> BN -> ReLU -> Conv -> BN
        out = self.conv1(x)
        out = self.bn1(out)
        out = F.relu(out, inplace=True)
        
        out = self.conv2(out)
        out = self.bn2(out)
        
        # Add skip connection (possibly transformed)
        out += self.shortcut(identity)
        
        # Final activation
        out = F.relu(out, inplace=True)
        
        return out

# Test the BasicBlock
print("=== Testing BasicBlock ===")

# Case 1: Same dimensions (no shortcut needed)
block1 = BasicBlock(64, 64, stride=1)
x1 = torch.randn(2, 64, 32, 32)
y1 = block1(x1)
print(f"Same dimensions: {x1.shape} -> {y1.shape}")

# Case 2: Downsampling (shortcut needed)
block2 = BasicBlock(64, 128, stride=2)
x2 = torch.randn(2, 64, 32, 32)
y2 = block2(x2)
print(f"Downsampling: {x2.shape} -> {y2.shape}")

# Count parameters
print(f"\nBlock1 parameters: {sum(p.numel() for p in block1.parameters()):,}")
print(f"Block2 parameters: {sum(p.numel() for p in block2.parameters()):,}")

### What Just Happened?

1. **Same dimensions (64→64)**: The shortcut is empty (`nn.Sequential()`), so the input passes through directly
2. **Downsampling (64→128, stride=2)**: The shortcut uses a 1×1 conv to match dimensions

Notice:
- Spatial dimensions halved: 32×32 → 16×16 (due to stride=2)
- Channels doubled: 64 → 128

This pattern (halve spatial, double channels) is common in CNNs!

### ✋ Try It Yourself: Exercise 1

Modify the BasicBlock to add a **dropout layer** after the first ReLU for regularization.

**Requirements:**
1. Add a `dropout` parameter to `__init__` (default 0.0)
2. Apply dropout after the first ReLU
3. Test with dropout=0.1

<details>
<summary>💡 Hint</summary>

Use `nn.Dropout2d(p=dropout)` for 2D spatial dropout. Remember to add it as a module attribute so it's properly registered!
</details>

In [ ]:
# YOUR CODE HERE: Implement BasicBlockWithDropout
class BasicBlockWithDropout(nn.Module):
    """BasicBlock with optional dropout regularization."""
    
    expansion = 1
    
    def __init__(self, in_channels: int, out_channels: int, stride: int = 1, dropout: float = 0.0):
        super().__init__()
        # TODO: Copy the BasicBlock implementation and add dropout
        pass
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # TODO: Implement forward with dropout
        pass

# Test your implementation
# block = BasicBlockWithDropout(64, 64, dropout=0.1)
# x = torch.randn(2, 64, 32, 32)
# y = block(x)
# print(f"Output shape: {y.shape}")

---

## Part 3: The Bottleneck Block - For Deeper Networks

For ResNet-50, ResNet-101, and ResNet-152, we use **Bottleneck blocks** instead. These are more efficient for deeper networks.

### Architecture:
```
Input (x, 256 channels)
    |
    +---> [1x1 Conv -> BN -> ReLU] (squeeze to 64 channels)
    |         |
    |         v
    |     [3x3 Conv -> BN -> ReLU] (process at 64 channels - cheaper!)
    |         |
    |         v  
    |     [1x1 Conv -> BN] (expand back to 256 channels)
    |                                                   
    +------------ (skip connection) -------------------+
                                                        |
                                                        v
                                                     [Add]
                                                        |
                                                        v
                                                     [ReLU]
                                                        |
                                                        v
                                                     Output (256 channels)
```

### ELI5: Why "Bottleneck"?

> Imagine you're moving water through pipes. A wide pipe can move a lot of water but costs more. 
>
> A bottleneck is like: Wide pipe → Narrow pipe → Wide pipe
>
> The narrow part ("bottleneck") is where the expensive 3×3 convolution happens, but with fewer channels, so it's cheaper! The 1×1 convolutions are like adapters that change the pipe width.

In [ ]:
class Bottleneck(nn.Module):
    """
    ResNet Bottleneck Block: 1x1 -> 3x3 -> 1x1 with skip connection.
    
    Used in ResNet-50, ResNet-101, ResNet-152.
    
    The expansion factor of 4 means the output has 4x the channels
    of the bottleneck layer.
    
    Args:
        in_channels: Number of input channels
        bottleneck_channels: Number of channels in the bottleneck (middle) layer
        stride: Stride for the 3x3 convolution
    """
    
    # Output channels = bottleneck_channels * expansion
    expansion = 4
    
    def __init__(
        self, 
        in_channels: int, 
        bottleneck_channels: int, 
        stride: int = 1
    ):
        super().__init__()
        
        out_channels = bottleneck_channels * self.expansion
        
        # 1x1 conv: squeeze channels
        self.conv1 = nn.Conv2d(
            in_channels, bottleneck_channels, 
            kernel_size=1, bias=False
        )
        self.bn1 = nn.BatchNorm2d(bottleneck_channels)
        
        # 3x3 conv: spatial processing (possibly with stride)
        self.conv2 = nn.Conv2d(
            bottleneck_channels, bottleneck_channels,
            kernel_size=3, stride=stride, padding=1, bias=False
        )
        self.bn2 = nn.BatchNorm2d(bottleneck_channels)
        
        # 1x1 conv: expand channels
        self.conv3 = nn.Conv2d(
            bottleneck_channels, out_channels,
            kernel_size=1, bias=False
        )
        self.bn3 = nn.BatchNorm2d(out_channels)
        
        # Shortcut connection
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        identity = x
        
        # Squeeze
        out = self.conv1(x)
        out = self.bn1(out)
        out = F.relu(out, inplace=True)
        
        # Process
        out = self.conv2(out)
        out = self.bn2(out)
        out = F.relu(out, inplace=True)
        
        # Expand
        out = self.conv3(out)
        out = self.bn3(out)
        
        # Add shortcut
        out += self.shortcut(identity)
        out = F.relu(out, inplace=True)
        
        return out

# Test the Bottleneck
print("=== Testing Bottleneck ===")

# Case 1: First block (needs shortcut to expand channels)
block1 = Bottleneck(64, 64, stride=1)  # 64 -> 256 channels
x1 = torch.randn(2, 64, 56, 56)
y1 = block1(x1)
print(f"First block: {x1.shape} -> {y1.shape}")

# Case 2: Subsequent block (no shortcut needed)
block2 = Bottleneck(256, 64, stride=1)  # 256 -> 256 channels
x2 = torch.randn(2, 256, 56, 56)
y2 = block2(x2)
print(f"Subsequent block: {x2.shape} -> {y2.shape}")

# Case 3: Downsampling block
block3 = Bottleneck(256, 128, stride=2)  # 256 -> 512 channels, size/2
x3 = torch.randn(2, 256, 56, 56)
y3 = block3(x3)
print(f"Downsampling block: {x3.shape} -> {y3.shape}")

# Compare parameters
print(f"\nBottleneck parameters: {sum(p.numel() for p in block2.parameters()):,}")
# Compare to equivalent BasicBlock (256 channels throughout)
equiv_basic = nn.Sequential(
    nn.Conv2d(256, 256, 3, padding=1),
    nn.Conv2d(256, 256, 3, padding=1)
)
print(f"Equivalent BasicBlock params: {sum(p.numel() for p in equiv_basic.parameters()):,}")

### What Just Happened?

The Bottleneck block:
1. **Squeezes** channels: 256 → 64 (1×1 conv)
2. **Processes** at reduced channels: 64 → 64 (3×3 conv, the expensive part)
3. **Expands** back: 64 → 256 (1×1 conv)

**Parameter comparison:**
- Bottleneck: ~70K parameters
- Equivalent BasicBlock: ~1.2M parameters (with 256 channels)

That's **17x fewer parameters** for the same layer connectivity!

---

## Part 4: Building Complete ResNet-18

Now let's assemble our blocks into a complete ResNet-18.

### ResNet-18 Architecture:

```
Layer              Output Size   Channels
─────────────────────────────────────────
Conv 7x7, stride 2    112×112       64
MaxPool 3x3, stride 2  56×56        64
─────────────────────────────────────────
Layer1: 2× BasicBlock   56×56       64
Layer2: 2× BasicBlock   28×28      128
Layer3: 2× BasicBlock   14×14      256
Layer4: 2× BasicBlock    7×7       512
─────────────────────────────────────────
AdaptiveAvgPool         1×1        512
FC                      1×1       1000 (ImageNet)
```

For CIFAR-10 (32×32 images), we'll modify the first layer to not downsample so aggressively.

In [ ]:
class ResNet(nn.Module):
    """
    ResNet implementation for CIFAR-10.
    
    Modified from standard ResNet:
    - First conv is 3x3 instead of 7x7 (images are 32x32, not 224x224)
    - No initial max pooling
    
    Args:
        block: Block type (BasicBlock or Bottleneck)
        layers: Number of blocks in each layer [layer1, layer2, layer3, layer4]
        num_classes: Number of output classes
    """
    
    def __init__(
        self,
        block: Type[Union[BasicBlock, Bottleneck]],
        layers: List[int],
        num_classes: int = 10
    ):
        super().__init__()
        
        self.in_channels = 64
        
        # Initial convolution - smaller for CIFAR-10
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        
        # ResNet layers
        self.layer1 = self._make_layer(block, 64, layers[0], stride=1)
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        
        # Classifier
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)
        
        # Weight initialization
        self._initialize_weights()
    
    def _make_layer(
        self, 
        block: Type[Union[BasicBlock, Bottleneck]], 
        channels: int, 
        num_blocks: int, 
        stride: int
    ) -> nn.Sequential:
        """Create a layer with multiple blocks."""
        
        layers = []
        
        # First block may downsample
        layers.append(block(self.in_channels, channels, stride))
        self.in_channels = channels * block.expansion
        
        # Remaining blocks maintain dimensions
        for _ in range(1, num_blocks):
            layers.append(block(self.in_channels, channels, stride=1))
        
        return nn.Sequential(*layers)
    
    def _initialize_weights(self):
        """Initialize weights using Kaiming initialization."""
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)
    
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        # Initial conv
        x = self.conv1(x)
        x = self.bn1(x)
        x = F.relu(x, inplace=True)
        
        # ResNet layers
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        # Classifier
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        
        return x


def resnet18(num_classes: int = 10) -> ResNet:
    """Create ResNet-18 model."""
    return ResNet(BasicBlock, [2, 2, 2, 2], num_classes)


def resnet34(num_classes: int = 10) -> ResNet:
    """Create ResNet-34 model."""
    return ResNet(BasicBlock, [3, 4, 6, 3], num_classes)


def resnet50(num_classes: int = 10) -> ResNet:
    """Create ResNet-50 model (with Bottleneck blocks)."""
    return ResNet(Bottleneck, [3, 4, 6, 3], num_classes)


# Test our ResNet-18
model = resnet18(num_classes=10)
print(f"ResNet-18 for CIFAR-10")
print(f"Total parameters: {sum(p.numel() for p in model.parameters()):,}")

# Test forward pass
x = torch.randn(4, 3, 32, 32)  # CIFAR-10 sized input
y = model(x)
print(f"\nInput shape: {x.shape}")
print(f"Output shape: {y.shape}")

# Show layer progression
print("\nLayer dimensions:")
x = torch.randn(1, 3, 32, 32)
x = F.relu(model.bn1(model.conv1(x)))
print(f"After conv1: {x.shape}")
x = model.layer1(x)
print(f"After layer1: {x.shape}")
x = model.layer2(x)
print(f"After layer2: {x.shape}")
x = model.layer3(x)
print(f"After layer3: {x.shape}")
x = model.layer4(x)
print(f"After layer4: {x.shape}")

---

## Part 5: Training on CIFAR-10

Now let's train our ResNet-18 on CIFAR-10 and see it in action!

In [ ]:
# Data transforms
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# Load CIFAR-10
trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_train
)
testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test
)

# DGX Spark optimization: Adaptive batch size based on available GPU memory
if torch.cuda.is_available():
    gpu_mem = torch.cuda.get_device_properties(0).total_memory / 1e9
    if gpu_mem > 100:  # DGX Spark with 128GB unified memory
        BATCH_SIZE = 256
    elif gpu_mem > 16:
        BATCH_SIZE = 128
    else:
        BATCH_SIZE = 64
else:
    BATCH_SIZE = 64

trainloader = DataLoader(
    trainset, batch_size=BATCH_SIZE, shuffle=True, 
    num_workers=4, pin_memory=True
)
testloader = DataLoader(
    testset, batch_size=BATCH_SIZE, shuffle=False, 
    num_workers=4, pin_memory=True
)

# Class names for visualization
classes = ('plane', 'car', 'bird', 'cat', 'deer', 
           'dog', 'frog', 'horse', 'ship', 'truck')

print(f"Training samples: {len(trainset):,}")
print(f"Test samples: {len(testset):,}")
print(f"Batch size: {BATCH_SIZE}")
print(f"Batches per epoch: {len(trainloader)}")

In [ ]:
def train_epoch(model, trainloader, criterion, optimizer, device):
    """Train for one epoch."""
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
    
    return running_loss / len(trainloader), 100. * correct / total


def evaluate(model, testloader, criterion, device):
    """Evaluate the model."""
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    
    with torch.no_grad():
        for inputs, targets in testloader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            
            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    
    return running_loss / len(testloader), 100. * correct / total


# Create model and move to GPU
model = resnet18(num_classes=10).to(device)

# Loss and optimizer
# Note: weight_decay=5e-4 is L2 regularization, a standard value for CIFAR-10 training
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)

# Learning rate scheduler - cosine annealing for smooth decay
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=10)

print(f"Model on device: {next(model.parameters()).device}")
print(f"GPU Memory allocated: {torch.cuda.memory_allocated()/1e9:.2f} GB")

In [ ]:
# Training loop - 10 epochs for demonstration
NUM_EPOCHS = 10

print("Starting training...")
print("="*60)

train_losses, train_accs = [], []
test_losses, test_accs = [], []

for epoch in range(NUM_EPOCHS):
    start_time = time.time()
    
    train_loss, train_acc = train_epoch(model, trainloader, criterion, optimizer, device)
    test_loss, test_acc = evaluate(model, testloader, criterion, device)
    
    scheduler.step()
    
    epoch_time = time.time() - start_time
    
    train_losses.append(train_loss)
    train_accs.append(train_acc)
    test_losses.append(test_loss)
    test_accs.append(test_acc)
    
    print(f"Epoch {epoch+1:2d}/{NUM_EPOCHS} | "
          f"Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}% | "
          f"Test Loss: {test_loss:.4f} | Test Acc: {test_acc:.2f}% | "
          f"Time: {epoch_time:.1f}s | LR: {scheduler.get_last_lr()[0]:.4f}")

print("="*60)
print(f"Final Test Accuracy: {test_accs[-1]:.2f}%")

In [ ]:
# Visualize training progress
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Loss plot
axes[0].plot(train_losses, label='Train')
axes[0].plot(test_losses, label='Test')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Training and Test Loss')
axes[0].legend()
axes[0].grid(True)

# Accuracy plot
axes[1].plot(train_accs, label='Train')
axes[1].plot(test_accs, label='Test')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy (%)')
axes[1].set_title('Training and Test Accuracy')
axes[1].legend()
axes[1].grid(True)

plt.tight_layout()
plt.show()

---

## Common Mistakes

### Mistake 1: Forgetting to call super().__init__()

```python
# ❌ Wrong - module won't work properly
class BadModule(nn.Module):
    def __init__(self):
        self.linear = nn.Linear(10, 10)

# ✅ Right - always call super first
class GoodModule(nn.Module):
    def __init__(self):
        super().__init__()  # ESSENTIAL!
        self.linear = nn.Linear(10, 10)
```

**Why:** Without `super().__init__()`, PyTorch can't track your parameters!

### Mistake 2: Not matching skip connection dimensions

```python
# ❌ Wrong - dimensions don't match, will crash
class BrokenBlock(nn.Module):
    def __init__(self, in_ch, out_ch, stride=2):
        super().__init__()
        self.conv = nn.Conv2d(in_ch, out_ch, 3, stride, 1)
    
    def forward(self, x):
        return self.conv(x) + x  # ERROR: x is wrong size!

# ✅ Right - use shortcut to match dimensions
class FixedBlock(nn.Module):
    def __init__(self, in_ch, out_ch, stride=2):
        super().__init__()
        self.conv = nn.Conv2d(in_ch, out_ch, 3, stride, 1)
        self.shortcut = nn.Conv2d(in_ch, out_ch, 1, stride)  # Match dimensions!
    
    def forward(self, x):
        return self.conv(x) + self.shortcut(x)
```

### Mistake 3: Using bias with BatchNorm

```python
# ❌ Wasteful - redundant bias
self.conv = nn.Conv2d(64, 64, 3, padding=1, bias=True)
self.bn = nn.BatchNorm2d(64)

# ✅ Better - BatchNorm has its own bias
self.conv = nn.Conv2d(64, 64, 3, padding=1, bias=False)
self.bn = nn.BatchNorm2d(64)
```

**Why:** BatchNorm normalizes and then applies its own learnable bias, making the conv bias redundant.

---

## Checkpoint

You've learned:
- ✅ How `nn.Module` tracks parameters and submodules
- ✅ The power of skip connections in ResNet
- ✅ BasicBlock for shallower networks (ResNet-18/34)
- ✅ Bottleneck for deeper networks (ResNet-50+)
- ✅ How to train a complete ResNet on CIFAR-10

---

## Challenge (Optional)

**Implement ResNet with Squeeze-and-Excitation (SE) blocks!**

SE blocks add channel attention to ResNet, improving accuracy by ~1%. The idea:

1. Global average pool to get channel-wise statistics
2. Two FC layers to learn channel importance
3. Multiply original features by learned weights

```
x -> [BasicBlock] -> feature -> GlobalAvgPool -> FC1 -> ReLU -> FC2 -> Sigmoid -> weights
                         |                                                          |
                         +-------------------------- × ----------------------------+
                                                     |
                                                   output
```

<details>
<summary>💡 Hint</summary>

```python
class SEBlock(nn.Module):
    def __init__(self, channels, reduction=16):
        super().__init__()
        self.fc1 = nn.Linear(channels, channels // reduction)
        self.fc2 = nn.Linear(channels // reduction, channels)
    
    def forward(self, x):
        b, c, _, _ = x.shape
        # Global average pool
        y = x.view(b, c, -1).mean(-1)
        # Channel attention
        y = F.relu(self.fc1(y))
        y = torch.sigmoid(self.fc2(y)).view(b, c, 1, 1)
        return x * y
```
</details>

---

## Further Reading

- [Deep Residual Learning (Original ResNet Paper)](https://arxiv.org/abs/1512.03385)
- [Identity Mappings in Deep Residual Networks](https://arxiv.org/abs/1603.05027)
- [PyTorch nn.Module Documentation](https://pytorch.org/docs/stable/generated/torch.nn.Module.html)
- [Squeeze-and-Excitation Networks](https://arxiv.org/abs/1709.01507)

In [ ]:
# Cleanup: Free GPU memory
import gc

del model, optimizer, scheduler
torch.cuda.empty_cache()
gc.collect()

print(f"GPU Memory after cleanup: {torch.cuda.memory_allocated()/1e9:.2f} GB")